In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from langchain.agents import AgentState

class CustomState(AgentState):
    favourite_colour: str

## Write to state

In [6]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def update_favourite_colour(favourite_colour: str, runtime: ToolRuntime) -> Command:
    """Update the favourite colour of the user in the state once they've revealed it."""
    return Command(update={
        "favourite_colour": favourite_colour, 
        "messages": [ToolMessage("Successfully updated favourite colour", tool_call_id=runtime.tool_call_id)]}
        )

In [7]:
from langchain.chat_models import init_chat_model
model = init_chat_model(model="meta-llama/llama-4-maverick-17b-128e-instruct",
                        model_provider="groq",
                        temperature = 0,
                        max_retries=3,
                        timeout=60,
                        max_tokens=300
                        )

In [8]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=model,
    tools=[update_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [9]:
from langchain.messages import HumanMessage

response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

In [10]:
from pprint import pprint

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='8f82d426-fd56-4152-9036-869bc421bf12'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'zbmkzt6nh', 'function': {'arguments': '{"favourite_colour":"green"}', 'name': 'update_favourite_colour'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 724, 'total_tokens': 757, 'completion_time': 0.037077537, 'completion_tokens_details': None, 'prompt_time': 0.019281317, 'prompt_tokens_details': None, 'queue_time': 0.054439532, 'total_time': 0.056358854}, 'model_name': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': 'fp_d2c1f7e199', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b3be4-11be-7e63-bff9-6e7deac35caa-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favour

In [11]:
response = agent.invoke(
    { 
        "messages": [HumanMessage(content="Hello, how are you?")],
        "favourite_colour": "green"
    }, # type: ignore
    {"configurable": {"thread_id": "10"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='543b4e94-9c16-47a2-959c-7eccd25f3975'),
              AIMessage(content="I'm doing well, thanks for asking. I'm a large language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or tasks you may have.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 725, 'total_tokens': 768, 'completion_time': 0.055442088, 'completion_tokens_details': None, 'prompt_time': 0.015123099, 'prompt_tokens_details': None, 'queue_time': 0.049321301, 'total_time': 0.070565187}, 'model_name': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': 'fp_d2c1f7e199', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b3be4-85b3-7680-8a8d-e10840e070a3-0', usage_metadata={'input_t

## Read state

In [12]:
@tool
def read_favourite_colour(runtime: ToolRuntime) -> str:
    """Read the favourite colour of the user from the state."""
    try:
        return runtime.state["favourite_colour"]
    except KeyError:
        return "No favourite colour found in state"

agent = create_agent(
    model=model,
    tools=[update_favourite_colour, read_favourite_colour],
    checkpointer=InMemorySaver(),
    state_schema=CustomState
)

In [13]:
response = agent.invoke(
    { "messages": [HumanMessage(content="My favourite colour is green")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='0793253e-5242-424b-ae53-f4379233f944'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '37sp8x5qa', 'function': {'arguments': '{"favourite_colour":"green"}', 'name': 'update_favourite_colour'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 755, 'total_tokens': 788, 'completion_time': 0.042515557, 'completion_tokens_details': None, 'prompt_time': 0.016004095, 'prompt_tokens_details': None, 'queue_time': 0.049224975, 'total_time': 0.058519652}, 'model_name': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': 'fp_d2c1f7e199', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b3be5-5069-7023-b16c-adc2de28b5c9-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favour

In [14]:
response = agent.invoke(
    { "messages": [HumanMessage(content="What's my favourite colour?")]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'favourite_colour': 'green',
 'messages': [HumanMessage(content='My favourite colour is green', additional_kwargs={}, response_metadata={}, id='0793253e-5242-424b-ae53-f4379233f944'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '37sp8x5qa', 'function': {'arguments': '{"favourite_colour":"green"}', 'name': 'update_favourite_colour'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 755, 'total_tokens': 788, 'completion_time': 0.042515557, 'completion_tokens_details': None, 'prompt_time': 0.016004095, 'prompt_tokens_details': None, 'queue_time': 0.049224975, 'total_time': 0.058519652}, 'model_name': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': 'fp_d2c1f7e199', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b3be5-5069-7023-b16c-adc2de28b5c9-0', tool_calls=[{'name': 'update_favourite_colour', 'args': {'favour